<a href="https://colab.research.google.com/github/Yard1/tune-sklearn-benchmark/blob/main/Tuning_Model_Benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install pycaret-nightly[full]
!pip install optuna
!pip install scikit-optimize
!pip install tune-sklearn ray[tune]
!pip install hpbandster ConfigSpace

In [6]:
from pycaret.datasets import get_data
data = get_data('juice')

,Id,Purchase,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
0,1,CH,237,1,1.75,1.99,0.00,0.0,0,0,0.500000,1.99,1.75,0.24,No,0.000000,0.000000,0.24,1
1,2,CH,239,1,1.75,1.99,0.00,0.3,0,1,0.600000,1.69,1.75,-0.06,No,0.150754,0.000000,0.24,1
2,3,CH,245,1,1.86,2.09,0.17,0.0,0,0,0.680000,2.09,1.69,0.40,No,0.000000,0.091398,0.23,1
3,4,MM,227,1,1.69,1.69,0.00,0.0,0,0,0.400000,1.69,1.69,0.00,No,0.000000,0.000000,0.00,1
4,5,CH,228,7,1.69,1.69,0.00,0.0,0,0,0.956535,1.69,1.69,0.00,Yes,0.000000,0.000000,0.00,0


In [7]:
from sklearn.model_selection import RandomizedSearchCV
from tune_sklearn import TuneSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from time import time
import pandas as pd

from pycaret.classification import *
cv = 5
n_iter = 100
s = setup(data, target = 'Purchase', session_id=42, silent = True, train_size=0.99, verbose=False)

x = get_config('X_train')
y = get_config('y_train')

def do_one_benchmark(seed):
  results = {}
  model = RandomForestClassifier(random_state=seed)
  pipeline = Pipeline([("empty", "passthrough"), ("estimator", model)])
  params = models(internal=True).loc["rf"]["Tune Grid"]
  pipeline_params = {f"estimator__{k}":v for k, v in params.items()}

  sklearn_search = RandomizedSearchCV(pipeline, pipeline_params, n_iter = n_iter, cv = cv, random_state = seed, n_jobs = -1, verbose=2)
  sklearn_search_no_pipe = RandomizedSearchCV(model, params, n_iter = n_iter, cv = cv, random_state = seed, n_jobs = -1, verbose=2)
  tune_search = TuneSearchCV(pipeline, pipeline_params, n_trials=n_iter,cv=cv,random_state=seed,n_jobs=1, verbose=1)
  tune_search_no_pipe =  TuneSearchCV(model, params, n_trials=n_iter,cv=cv,random_state=seed,n_jobs=1, verbose=1)
  
  time_start = time()
  sklearn_search_no_pipe.fit(x,y)
  time_end = time()-time_start
  results["sklearn"] = time_end

  time_start = time()
  tune_search_no_pipe.fit(x,y)
  time_end = time()-time_start
  results["tune-sklearn"] = time_end
  return results

In [8]:
seeds = [1, 42, 1337, 24, 7331]
results = []
for seed in seeds:
  results.append(do_one_benchmark(seed))
pd.DataFrame(results, index=seeds)

,sklearn,tune-sklearn
1,133.324383,177.800076
42,132.011034,208.051172
1337,135.047765,183.514914
24,140.401911,202.957655
7331,145.438902,192.463988
